In [162]:
%pip install -U langchain-community


Note: you may need to restart the kernel to use updated packages.


In [163]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone

from pinecone import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [164]:
PINECONE_API_ENV="us-east-1"
PINECONE_API_KEY="9e0943d2-1669-4389-9af3-f0b40bd3aab7"

In [165]:
%pip install pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [166]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("healthbot")

In [167]:
index.upsert(
    vectors=[
        {
            "id": "vec1", 
            "values": [0.1] * 384, 
            "metadata": {"genre": "drama"}
        }, {
            "id": "vec2", 
            "values": [0.2] * 384, 
            "metadata": {"genre": "action"}
        }, {
            "id": "vec3", 
            "values": [0.3] * 384, 
            "metadata": {"genre": "drama"}
        }, {
            "id": "vec4", 
            "values": [0.4] * 384, 
            "metadata": {"genre": "action"}
        }
    ],
    namespace= "ns1"
)


{'upserted_count': 4}

In [168]:
index.query(
    namespace="ns1",
    vector=[0.3] * 384,
    top_k=2,
    include_values=True,
    include_metadata=True,
    filter={"genre": {"$eq": "action"}}
)

{'matches': [{'id': 'vec2',
              'metadata': {'genre': 'action'},
              'score': 0.999999762,
              'values': [0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                         0.2,
                   

In [169]:
# Extract pdf data
from unittest import loader


def load_pdf(data):
    directory_loader = DirectoryLoader(data, 
                                       glob="*.pdf",
                                       loader_cls=PyPDFLoader)
    
    documents = directory_loader.load()

    

In [170]:
extracted_data = DirectoryLoader("data/")

In [171]:
extracted_data


In [172]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [173]:
import os
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def load_pdf(data):
    # Print the current working directory
    print("Current working directory:", os.getcwd())  
    
    # Check if the directory exists
    if not os.path.exists(data):
        raise FileNotFoundError(f"Directory not found: '{data}'")
    if not os.path.isdir(data):
        raise ValueError(f"Expected a directory, but got: '{data}'")
    
    # Load the documents
    directory_loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = directory_loader.load()
    return documents

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

# Check current working directory
print("Current working directory:", os.getcwd())

# Load the PDFs
extracted_data = load_pdf("../data/")  # Ensure this path is correct relative to your script

# Split the text into chunks
text_chunks = text_split(extracted_data)
print(len(text_chunks))


Current working directory: c:\Users\MIKE\Desktop\healthylivingbot\research
Current working directory: c:\Users\MIKE\Desktop\healthylivingbot\research
14040


In [174]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [175]:
embeddings = download_hugging_face_embeddings()

c:\Users\MIKE\miniconda3\envs\mchatbot\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [176]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [177]:
query_result = embeddings.embed_query("Hello world")
print("length", len(query_result))

length 384


In [178]:
query_result

[-0.03447727486491203,
 0.03102317824959755,
 0.006734970025718212,
 0.026108985766768456,
 -0.03936202451586723,
 -0.16030244529247284,
 0.06692401319742203,
 -0.006441489793360233,
 -0.0474504791200161,
 0.014758856035768986,
 0.07087527960538864,
 0.05552763119339943,
 0.019193334504961967,
 -0.026251312345266342,
 -0.01010954286903143,
 -0.02694045566022396,
 0.022307461127638817,
 -0.022226648405194283,
 -0.14969263970851898,
 -0.017493007704615593,
 0.00767625542357564,
 0.05435224249958992,
 0.0032543970737606287,
 0.031725890934467316,
 -0.0846213847398758,
 -0.02940601296722889,
 0.05159561336040497,
 0.04812406003475189,
 -0.0033148222137242556,
 -0.058279167860746384,
 0.04196927323937416,
 0.022210685536265373,
 0.1281888335943222,
 -0.022338971495628357,
 -0.011656315997242928,
 0.06292839348316193,
 -0.032876335084438324,
 -0.09122604131698608,
 -0.031175347045063972,
 0.0526994913816452,
 0.04703482985496521,
 -0.08420311659574509,
 -0.030056199058890343,
 -0.02074483036

In [179]:
%pip install pinecone.Client
%pip install langchain_pinecone

Note: you may need to restart the kernel to use updated packages.
  Using cached pinecone_client-3.2.2-py3-none-any.whl.metadata (16 kB)
Using cached pinecone_client-3.2.2-py3-none-any.whl (215 kB)
  Attempting uninstall: pinecone-client
    Found existing installation: pinecone-client 4.1.1
    Uninstalling pinecone-client-4.1.1:
      Successfully uninstalled pinecone-client-4.1.1
Note: you may need to restart the kernel to use updated packages.


In [180]:
import os
from pinecone import Pinecone

os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

pinecone_instance = Pinecone(api_key=PINECONE_API_KEY, environment='us-east-1') #I dont think this line does anything but sijaitoa just incase

index_name = "healthbot"

# Create a PineconeVectorStore instance with the Pinecone instance and index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [181]:
%pip install pinecone_client

Note: you may need to restart the kernel to use updated packages.


In [183]:
import os
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

# Set environment variables for your API keys
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

# Initialize the Pinecone index name and embeddings
index_name = "healthbot"
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")  # Adjust the model if necessary to ensure it outputs 384-dimensional vectors

# Initialize the PineconeVectorStore from an existing index
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

# Perform a similarity search with the query
query = "What are Allergies"
docs = vectorstore.similarity_search(query, k=3)

# Print the results
print("Result", docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which co

In [186]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""





In [187]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [188]:
import os

# Print current working directory
print("Current working directory:", os.getcwd())

# Check if the file exists
model_path = "model/llama-2-7b-chat.ggmlv3.q4_0.bin"
print("Model exists:", os.path.exists(model_path))

# Load the model if the path is correct
if os.path.exists(model_path):
    llm = CTransformers(model=model_path, model_type="llama", config={'max_new_tokens': 512, 'temperature': 0.8})
else:
    print("Model path is incorrect. Please check the file path.")


Current working directory: c:\Users\MIKE\Desktop\healthylivingbot\research
Model exists: True


In [189]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})


In [ ]:
%pip install pinecone-client --upgrade

  Using cached pinecone_client-4.1.1-py3-none-any.whl.metadata (16 kB)
Using cached pinecone_client-4.1.1-py3-none-any.whl (216 kB)
  Attempting uninstall: pinecone-client
    Found existing installation: pinecone-client 3.2.2
    Uninstalling pinecone-client-3.2.2:
      Successfully uninstalled pinecone-client-3.2.2
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-pinecone 0.1.1 requires pinecone-client<4.0.0,>=3.2.2, but you have pinecone-client 4.1.1 which is incompatible.


In [190]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [191]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

c:\Users\MIKE\miniconda3\envs\mchatbot\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Response :  The concept of "right" and "want" can be complex and subjective, as it depends on individual perspectives and cultural norms. However, some general principles that may guide what is considered right and doing what one wants include:
* Respect for ethical and moral principles, such as honesty, fairness, and compassion.
* Consideration of the well-being and rights of others, including their autonomy and dignity.
* Acknowledgment of responsibilities and obligations towards others, such as fulfilling commitments and duties.
* Reflection on personal values and goals, and how they align with one's actions and decisions.
* Awareness of cultural norms and social expectations, and how they shape individual beliefs and behaviors.
It is important to recognize that what is considered right and doing what one wants can vary greatly across different contexts and cultures, and it is essential to approach these questions with sensitivity and respect for diversity.
